# Data multiplication

In case if you do no thave enough training data you can multiply it by rotating or adding coloured background




## Colouring background


Gets images from files folder. Returns to files folder

In [1]:
import cv2
import os
import numpy as np

In [2]:
toprocess = "toprocess/"
root = "./../files/train/"
yes_no_pathes = ["yes/","no/"]

In [3]:
BG_COLOR = 209
BG_SIGMA = 5
MONOCHROME = 1

def blank_image(width=256, height=256, background=BG_COLOR):
    img = np.full((height, width, MONOCHROME), background, np.uint8)
    return img


def add_noise(img, sigma=BG_SIGMA):
    width, height, ch = img.shape
    n = noise(width, height, sigma=sigma)
    img = img + n
    return img.clip(0, 255)


def noise(width, height, ratio=1, sigma=BG_SIGMA):
    mean = 0
    assert width % ratio == 0, "Can't scale image with of size {} and ratio {}".format(width, ratio)
    assert height % ratio == 0, "Can't scale image with of size {} and ratio {}".format(height, ratio)

    h = int(height / ratio)
    w = int(width / ratio)

    result = np.random.normal(mean, sigma, (w, h, MONOCHROME))
    if ratio > 1:
        result = cv2.resize(result, dsize=(width, height), interpolation=cv2.INTER_LINEAR)
    return result.reshape((width, height, MONOCHROME))


def texture(image, sigma=BG_SIGMA, turbulence=2):
    result = image.astype(float)
    cols, rows, ch = image.shape
    ratio = cols
    while not ratio == 1:
        result += noise(cols, rows, ratio, sigma=sigma)
        ratio = (ratio // turbulence) or 1
    cut = np.clip(result, 0, 255)
    return cut.astype(np.uint8)

In [4]:
bg1 = add_noise(texture(blank_image(background=230), sigma=4), sigma=10)
bg2 = add_noise(texture(blank_image(background=20), sigma=8), sigma=20)
bg3 = add_noise(texture(blank_image(background=230), sigma=4), sigma=1)
bg4 = add_noise(texture(blank_image(background=230), sigma=10), sigma=5)

In [5]:
def add_background(path, name):
    image = cv2.imread(path + toprocess+name, cv2.IMREAD_UNCHANGED)
    image = cv2.resize(image, (256,256))
    trans_mask = image[:, :, 3] == 0
    # replace areas of transparency with white and not transparent
    image[trans_mask] = bg1[trans_mask]
    nimage = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    cv2.imwrite(path + toprocess + name.replace('.', f'_{1}.'), nimage)
    image[trans_mask] = bg2[trans_mask]
    nimage = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    cv2.imwrite(path+toprocess  + name.replace('.', f'_{2}.'), nimage)
    image[trans_mask] = bg3[trans_mask]
    nimage = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    cv2.imwrite(path+toprocess  + name.replace('.', f'_{3}.'), nimage)
    image[trans_mask] = bg4[trans_mask]
    nimage = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    cv2.imwrite(path +toprocess + name.replace('.', f'_{4}.'), nimage)
    


In [6]:

for yes_no in yes_no_pathes:
    path = root+yes_no+toprocess
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.split(".")[-1]=="png"]
    for f in files:
        add_background(root+yes_no, f)

## Rotating images

In [7]:
import cv2
import os
import numpy as np

In [8]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

def add_rotations(path, name, yes_no):
    image = cv2.imread(path+name, cv2.IMREAD_COLOR)
    for i in range(0, 271, 90):
        nimage = rotate_image(image, i)
        cv2.imwrite(root+yes_no +name.replace('.', f'_{i}.'),nimage)
        

In [9]:
for yes_no in yes_no_pathes:
    path = root+yes_no+toprocess
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    for name in files:
        add_rotations(path, name, yes_no)